# <div align="center" style="color: #ff5733;">TSA Closure Second Notification</div>

In [24]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

# Ask Balasubramani to refresh risk_credit_mis.tsa_account_closure table.
**Check if that is refreshed when Balasubramani confirms it**

In [25]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID 141e4d41-9c6e-4dce-8cb6-62676b961bbc successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-07-07,595391


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [26]:
d1 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\May24&June24 Ongoing Batch\SecondNotification_files\Type 1 & 2.xlsx")
d1.shape

(8132, 2)

In [27]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()


The unique count of customer_id is:	8130


,Customer_Id,ACTION_TYPE
0,2196994,TYPE 1
1,2441154,TYPE 1
2,2435167,TYPE 1
3,2418183,TYPE 1
4,2433983,TYPE 1


# Read the data from the second sheet for Type3 data provided by Marvin

In [28]:
d2 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\May24&June24 Ongoing Batch\SecondNotification_files\Type 3.xlsx")
d2.shape

(951, 2)

In [29]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	951


,Customer_Id,ACTION_TYPE
0,1371405,TYPE 3
1,1477514,TYPE 3
2,1852692,TYPE 3
3,2292763,TYPE 3
4,2035788,TYPE 3


In [30]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(9083, 2)

In [31]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    8107
TYPE 3     951
TYPE 2      25
Name: count, dtype: int64

In [35]:
result[result['Customer_Id'] == '2256481']

,Customer_Id,ACTION_TYPE
8188,2256481,TYPE 3


In [33]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9083 entries, 0 to 9082
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  9083 non-null   int64 
 1   ACTION_TYPE  9083 non-null   object
dtypes: int64(1), object(1)
memory usage: 142.0+ KB


In [34]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9083 entries, 0 to 9082
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  9083 non-null   object
 1   ACTION_TYPE  9083 non-null   object
dtypes: object(2)
memory usage: 142.0+ KB


In [36]:
result.to_csv("temp.csv", index = False)

In [37]:
a = tuple(result['Customer_Id'])
a[:5]

('2196994', '2441154', '2435167', '2418183', '2433983')

In [38]:
d = '2024-07-07'   ### Need to change this to D-1 everytime we run this code ###
# Get the current date
current_date = pd.Timestamp.now().date()
d_minus_1_days = current_date - pd.DateOffset(days=1)
# Calculate D-60 days
d_minus_60_days = '2024-05-08'
# Calculate D-180 days
d_minus_180_days = '2024-01-09'

print("D-1 day from current date", d_minus_1_days )
print("D-60 day from the current date - 1:", d_minus_60_days)
print("D-180 day from the current date -1:", d_minus_180_days)

d_minus_1_days_string = d_minus_1_days.strftime('%Y-%m-%d')
print("D-1 day from the current date:", d_minus_1_days_string)

d_minus_60_days_string = d_minus_60_days
print("D-60 day from the current date:", d_minus_60_days_string)

d_minus_180_days_string = d_minus_180_days
print("D-180 day from the current date:", d_minus_180_days_string)

D-1 day from current date 2024-07-07 00:00:00
D-60 day from the current date - 1: 2024-05-08
D-180 day from the current date -1: 2024-01-09
D-1 day from the current date: 2024-07-07
D-60 day from the current date: 2024-05-08
D-180 day from the current date: 2024-01-09


The date should be D-1? If the date is not D-1 then ask Bala to refresh the table.

Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [39]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}' ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id not in {a}
)
select * from w_tsa;
"""

dfd = client.query(sq).to_dataframe()
dfd.shape

(549152, 29)

In [41]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id not in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '{d_minus_60_days_string}')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '{d_minus_60_days_string}')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
            and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
            and (Active_Term_Deposit_flag = 0)
            and TSA_Opening_date < date '{d_minus_180_days_string}'                                     ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and (Last_TSA_Credit_Date < date '{d_minus_180_days_string}'                                 ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and Last_TSA_Debit_Date < date '{d_minus_180_days_string}'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '{d_minus_180_days_string}' )  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and tsa_status = 'Active'
            and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
            and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
)
"""


In [42]:
print(sq)


with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-07-07'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id not in ('2196994', '2441154', '2435167', '2418183', '2433983', '2439130', '2418300', '2422147', '2418098', '2440531', '2417980', '2419103', '2436954', '2437152', '2401242', '2436176', '2425670', '2424811', '2437727', '2416976', '2409334', '2429740', '2412912', '2416940', '2411862', '2148884', '2440500', '2422303', '2409064', '2426753', '2416725', '2439106', '2413105', '2404666', '1116283', '2410336', '2405168', '24397

In [43]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 95f6f473-17e6-45ae-8ed0-d49c2d127062 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [44]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
132923,1954123,2023-03-21,60819541230006,0.0,Active,0.00,0.00,0.0,0.0,0,2024-06-30,2024-07-02,N,1,21504.26,1,0,0,0,0,2024-06-21,2024-07-01,NaT,NaN,2024-07-07,Batch_1,July,0,0,NaT,NaT,NO ACTION
521890,2611722,2024-06-26,60826117220001,0.0,Active,0.00,0.00,0.0,0.0,0,1970-01-01,1970-01-01,N,1,6327.30,1,0,0,0,0,2024-06-26,NaT,NaT,NaN,2024-07-07,Batch_1,July,0,0,NaT,NaT,NO ACTION
294274,1382826,2022-05-21,60813828260001,0.0,Active,0.29,12.96,0.0,0.0,0,2024-04-30,2022-06-30,N,0,0.00,0,1,0,0,0,2022-11-15,2024-07-07,NaT,NaN,2024-07-07,Batch_1,July,1,0,2022-08-11,NaT,NO ACTION
124582,1607630,2023-04-24,60816076300009,0.0,Active,0.02,2.41,0.0,0.0,0,2023-09-07,2023-09-07,N,0,0.00,0,1,0,0,1,2023-09-07,2024-07-07,NaT,NaN,2024-07-07,Batch_1,July,1,3,2023-04-24,2023-12-07,NO ACTION
116362,2512770,2024-05-13,60825127700006,0.0,Active,0.00,0.00,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.00,0,0,0,1,0,2024-05-17,NaT,NaT,NaN,2024-07-07,Batch_1,July,0,0,NaT,NaT,NO ACTION
385739,1741410,2022-10-29,60817414100001,0.0,Active,0.00,0.00,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.00,0,1,0,0,0,2022-12-31,NaT,NaT,NaN,2024-07-07,Batch_1,July,0,0,NaT,NaT,NO ACTION
293681,1457223,2022-07-10,60814572230001,0.0,Active,0.00,0.00,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.00,0,1,0,0,0,2022-07-10,NaT,NaT,NaN,2024-07-07,Batch_1,July,0,0,NaT,NaT,NO ACTION
221230,2574019,2024-06-13,60825740190001,0.0,Active,0.00,0.00,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.00,0,0,0,1,0,2024-06-13,NaT,NaT,NaN,2024-07-07,Batch_1,July,0,0,NaT,NaT,NO ACTION
71235,1022063,2021-04-03,60810220630004,0.0,Active,887.59,49931.15,0.0,0.0,0,2024-06-30,2024-06-30,N,0,0.00,0,0,0,0,0,2024-07-02,2024-07-07,NaT,NaN,2024-07-07,Batch_1,July,0,0,NaT,NaT,NO ACTION
536856,1740940,2022-10-28,60817409400008,0.0,Active,0.00,0.00,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.00,0,2,0,0,1,2022-10-31,NaT,NaT,NaN,2024-07-07,Batch_1,July,2,0,2022-11-30,NaT,NO ACTION


In [45]:
df.shape

(549156, 32)

In [46]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
NO ACTION    533381
TYPE 1        14313
TYPE 3         1432
TYPE 2           30
Name: count, dtype: int64

In [49]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\May24&June24 Ongoing Batch\2024018_TSA_Account_Closure_firstNotificationexcludingattachment.xlsx", sheet_name='firstNotificationexclattachment', index = False)